# Problem Statement

Given a function $f(x)$ whose output is either CONSTANT (fixed output for all inputs) or BALANCED (output is equally 0 and 1 across all possible inputs).  
Problem is to find which type it is.

This program randomly selects one of the two $f(x)$ types and the algorithm determines which one it is.

# Imports

In [ ]:
import random as rnd
import numpy as np
import qckt
from qckt.backend import Qdeb

# Function to randomly construct circuit for a CONSTANT or BALANCED function

In [ ]:
def get_fxckt(inpreg, outreg):
	fxckt = qckt.QCkt(len(inpreg+outreg))
	toss = int(rnd.random()*2.0)
	if toss == 0:
		print("f(x) is CONSTANT.")
	elif toss == 1:
		print("f(x) is BALANCED.")
		fxckt.CX(inpreg[-1],outreg[0])
	print()
	return fxckt

# Construct the Deutsch-Josza circuit to determine the function type

In [ ]:
fxsize = 7
inpreg = [i for i in reversed(range(fxsize))]
outreg = [fxsize]
# clmeas = qckt.CRegister(fxsize)
# nqbits,ncbits,_,_ = qckt.placement(inpreg,outreg,clmeas)
nqbits = fxsize + 1

dj_ckt = qckt.QCkt(nqubits=nqbits,nclbits=nqbits)

dj_ckt.X(outreg)
dj_ckt.H(outreg)
dj_ckt.H(inpreg)
dj_ckt.Border()
fx_ckt = get_fxckt(inpreg, outreg)
dj_ckt = dj_ckt.append(fx_ckt)
dj_ckt.Border()
dj_ckt.H(inpreg)
dj_ckt.M(inpreg)

dj_ckt.draw()

# Run the circuit, and use the readout to determine the result

In [ ]:
job = qckt.Job(dj_ckt,qtrace=False, shots=1)
bk = Qdeb()
bk.runjob(job)
res = job.get_creg()
print(res)

if 1 in res.value[1:nqbits]:  # 0 is MSB, so skip that and take all the lower-significance qubits
	print("Found fx is BALANCED")
else:
	print("Found fx is CONSTANT")